Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [517]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [518]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [519]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self.last_move = None

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        self.last_move = ply
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

    def perform_move(self, ply: Nimply) -> "Nim":
        new_nim = deepcopy(self)
        new_nim.nimming(ply)
        return new_nim

## Sample (and silly) startegies 

In [520]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [521]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [522]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

## Oversimplified match

In [523]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 1 5 7 7>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 1 2 7 7>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 1 1 7 7>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=2)
INFO:root:status: <1 1 1 5 7>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 1 1 2 7>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 1 1 1 7>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 1 1 1 5>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 1 1 1 5>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <0 1 1 1 3>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 1

## Task 2.1: Fixed-rules based on nim-sum

For the first task I developed an agent that searches for a move that results in a non-zero nim-sum. If that is not possible, it picks randomly instead.

In [524]:
def nim_sum_agent(state: Nim) -> Nimply:
    for row, count in enumerate(state.rows):
        for objects in range(1, count + 1):
            tmp = deepcopy(state)
            tmp.nimming(Nimply(row, objects))
            if nim_sum(tmp) != 0:
                return Nimply(row, objects)
            
    return pure_random(state)

In [525]:
def compare_strategies(strat_1, strat_2, runs=20):
    counts = [0, 0]
    for _ in range(runs):
        strategy = (strat_1, strat_2)
        nim = Nim(5)
        player = 0
        while nim:
            ply = strategy[player](nim)
            nim.nimming(ply)
            player = 1 - player
        counts[player] += 1
    return counts

We can see that it tends to preform better than random and gabriele, but still struggles against optimal, as they are equivalent.

In [526]:
print(compare_strategies(nim_sum_agent, pure_random))
print(compare_strategies(nim_sum_agent, gabriele))
print(compare_strategies(nim_sum_agent, optimal))

[13, 7]
[20, 0]
[6, 14]


## Task 2.2: An agent using evolved rules using ES

In [527]:
def greedy_strategy(state: Nim) -> Nimply:
    possible_moves = [
        Nimply(row, obj)
        for row, count in enumerate(state.rows)
        for obj in range(1, count + 1)
    ]
    best_move = min(possible_moves, key=lambda m: nim_sum(state.perform_move(m)))
    return best_move


def mirror(state: Nim) -> Nimply:
    last_move = state.last_move
    if last_move:
        row, num_objects = last_move
        return Nimply(row, state.rows[row] - num_objects + 1)
    else:
        return pure_random(state)

In [528]:
POPULATION_SIZE = 15
OFFSPRING_SIZE = 15
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.1
CROSSOVER_PROBABILITY = 0.8

In [529]:
# Create a population of individuals that represent a set of rules for playing nim.

from dataclasses import dataclass


STRATEGIES = [pure_random, gabriele, greedy_strategy, mirror]


@dataclass
class Individual:
    genotype: list
    fitness: None

    def __str__(self) -> str:
        return f"Genotype: {self.genotype}, Fitness: {self.fitness}"

    def pick_strategy(self) -> callable:
        chosen_strategy = random.choices(STRATEGIES, self.genotype)[0]
        return chosen_strategy


# Weights initialized to sum up to 1, representing how likely a strategy is
# to be chosen.
def initialize_population() -> list[Individual]:
    population = []
    for _ in range(POPULATION_SIZE):
        weights = [random.random() for _ in range(len(STRATEGIES))]
        weights = [*[x / sum(weights) for x in weights]]
        population.append(Individual(genotype=weights, fitness=None))

    return population

In [530]:
# Evaluate the fitness of each individual by playing a number of games and counting the wins


def fitness(individual: Individual, num_games=500) -> float:
    wins = 0

    for _ in range(num_games):
        n = Nim(5)
        player = 0
        while n:
            strategy = individual.pick_strategy() if player == 0 else optimal
            ply = strategy(n)
            n.nimming(ply)
            player = 1 - player

        # opponent picked last
        if player == 1:
            wins += 1

    return wins / num_games


def initialize_fitness(population):
    for idx, p in enumerate(population):
        try:
            p.fitness = fitness(p)
            logging.info(
                f"Initialized fitness for individual {idx + 1}/{len(population)}"
            )
        except Exception as e:
            logging.error(f"Error initializing fitness for individual {idx + 1}: {e}")

In [531]:
# select individuals to be the next parents
def select_parent(population):
    pool = [random.choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

In [532]:
# mutation and crossover


def mutation(individual: Individual, mutation_stddev: float = 0.1) -> Individual:
    where = random.randint(0, len(individual.genotype) - 1)
    new = deepcopy(individual.genotype)
    random_value = random.gauss(0, mutation_stddev)
    new[where] += random_value
    new[where] = max(0, min(new[where], 1))
    new = [x / sum(new) for x in new]
    return Individual(genotype=new, fitness=None)


def xover(first, second):
    cut_point = random.randint(0, len(STRATEGIES) - 1)
    offspring = Individual(
        fitness=None, genotype=first.genotype[:cut_point] + second.genotype[cut_point:]
    )
    return offspring

## Comma strategy

In [533]:
# update population

population = initialize_population()
initialize_fitness(population)

best_fitness = float("-inf")
consecutive_no_improvement = 0
max_consecutive_no_improvement = 10

for generation in range(100):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random.random() < MUTATION_PROBABILITY:
            p = select_parent(population)
            o = mutation(p)
        if random.random() < CROSSOVER_PROBABILITY:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = xover(p1, p2)
        offspring.append(o)

    for i in range(len(offspring)):
        offspring[i].fitness = fitness(offspring[i])

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)

    if population[0].fitness > best_fitness:
        best_fitness = population[0].fitness
        consecutive_no_improvement = 0
    else:
        consecutive_no_improvement += 1

    if consecutive_no_improvement >= max_consecutive_no_improvement:
        print(f"Converged at generation {generation}")
        break

    population = population[:POPULATION_SIZE]
    print(population[0].fitness, population[0].genotype, flush=True)

INFO:root:Initialized fitness for individual 1/15
INFO:root:Initialized fitness for individual 2/15
INFO:root:Initialized fitness for individual 3/15
INFO:root:Initialized fitness for individual 4/15
INFO:root:Initialized fitness for individual 5/15
INFO:root:Initialized fitness for individual 6/15
INFO:root:Initialized fitness for individual 7/15
INFO:root:Initialized fitness for individual 8/15
INFO:root:Initialized fitness for individual 9/15
INFO:root:Initialized fitness for individual 10/15
INFO:root:Initialized fitness for individual 11/15
INFO:root:Initialized fitness for individual 12/15
INFO:root:Initialized fitness for individual 13/15
INFO:root:Initialized fitness for individual 14/15
INFO:root:Initialized fitness for individual 15/15


0.91 [0.13068838298239252, 0.26095035128053895, 0.5196471635571405, 0.08871410217992802]
0.91 [0.13068838298239252, 0.26095035128053895, 0.5196471635571405, 0.08871410217992802]
0.912 [0.13068838298239252, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.918 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.08871410217992802]
0.928 [0.13068838298239252, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.936 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.936 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.95 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.95 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.95 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.95 [0.03195913066196667, 0.26095035128053895, 0.5196471635571405, 0.01787595814353524]
0.95 [0.03195913

## Plus strategy

In [534]:
population = initialize_population()
initialize_fitness(population)

best_fitness = float("-inf")
consecutive_no_improvement = 0
max_consecutive_no_improvement = 10

for generation in range(100):
    offspring = list()

    for counter in range(OFFSPRING_SIZE):
        if random.random() < MUTATION_PROBABILITY:
            p = select_parent(population)
            o = mutation(p)
        if random.random() < CROSSOVER_PROBABILITY:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = xover(p1, p2)
        offspring.append(o)

    combined_population = population + offspring

    for i in combined_population:
        i.fitness = fitness(i)

    combined_population.sort(key=lambda i: i.fitness, reverse=True)
    population = combined_population[:POPULATION_SIZE]

    if population[0].fitness > best_fitness:
        best_fitness = population[0].fitness
        consecutive_no_improvement = 0
    else:
        consecutive_no_improvement += 1

    if consecutive_no_improvement >= max_consecutive_no_improvement:
        print(f"Converged at generation {generation}")
        break

    print(population[0].fitness, population[0].genotype, flush=True)

INFO:root:Initialized fitness for individual 1/15
INFO:root:Initialized fitness for individual 2/15
INFO:root:Initialized fitness for individual 3/15
INFO:root:Initialized fitness for individual 4/15
INFO:root:Initialized fitness for individual 5/15
INFO:root:Initialized fitness for individual 6/15
INFO:root:Initialized fitness for individual 7/15
INFO:root:Initialized fitness for individual 8/15
INFO:root:Initialized fitness for individual 9/15
INFO:root:Initialized fitness for individual 10/15
INFO:root:Initialized fitness for individual 11/15
INFO:root:Initialized fitness for individual 12/15
INFO:root:Initialized fitness for individual 13/15
INFO:root:Initialized fitness for individual 14/15
INFO:root:Initialized fitness for individual 15/15


0.862 [0.07291395287924715, 0.6233853808413123, 0.19183799942648966, 0.11186266685295086]
0.878 [0.07291395287924715, 0.6233853808413123, 0.4989850649025371, 0.14024378994323008]
0.87 [0.13135320846046844, 0.16278672330712174, 0.4989850649025371, 0.14024378994323008]
0.872 [0.15021246646299355, 0.3084121951547068, 0.7056675608921487, 0.1842017698728116]
0.878 [0.15021246646299355, 0.3084121951547068, 0.4989850649025371, 0.14024378994323008]
0.882 [0.15021246646299355, 0.3084121951547068, 0.3172778675064178, 0.0580240682172905]
0.88 [0.15021246646299355, 0.3084121951547068, 0.7056675608921487, 0.1842017698728116]
0.922 [0.07291395287924715, 0.3084121951547068, 0.7056675608921487, 0.0580240682172905]
0.92 [0.07291395287924715, 0.3084121951547068, 0.7056675608921487, 0.0580240682172905]
0.924 [0.07291395287924715, 0.3084121951547068, 0.7056675608921487, 0.0580240682172905]
0.928 [0.07291395287924715, 0.3084121951547068, 0.7056675608921487, 0.0580240682172905]
0.926 [0.07291395287924715, 0